GRPO的最大化期望式子为：
$$
\nabla_{\theta}\mathcal{J}_{\mathcal{A}}(\theta)=\mathbb{E}[(q, o) \sim \mathcal{D}]\left(\frac{1}{|o|}\sum\limits_{t=1}^{|o|}GC_{\mathcal{A}}(q, o, t, \pi_{rf}) \nabla_{\theta}\log{\pi_{\theta}(o_t|q, o<t)} \right)
$$
其中期望内部的表示数据和输出服从抽取数据的分布，$\pi_{rf}$ 是奖励模型，如果在一般的任务中其实就是 $\text{env}$ 给的 reward，GC的计算公式如下：
$$
GC_{GRPO}(q,o,t,\pi_{\theta_{rm}}) = \hat{A}_{i,t} + \beta \left(\frac{\pi_{ref}(o_{i,t} | o_{i, <t})}{\pi_{\theta}(o_{i,t} | o_{i, <t})} - 1 \right)
$$
其中 $\hat{A}_{i,t}$ 的对于不同种类的监督方式有不同的计算方法：

- 对于结果监督，G组数据中的每一组数据给一个reward，因此有 $\hat{A}_{i,t}=\frac{r_i - mean(r)}{std(r)}$
- 对于过程监督，G组数据中的每一组中的每一个action都对应一个reward，因此有 $\tilde{r}_{i}^{index(j)} = \frac{r_i^{index(j)}-\text{mean}(R)}{\text{std}(R)}$，以及 $\hat{A}_{i,t}=\sum_{index(j) \geq t}r_i^{index(j)}$

整体的优势的计算以及整体期望计算非常简单。


In [ ]:
import torch
tmp_state_dict = torch.load("./model_parameters/ppo.pt")
print(tmp_state_dict.keys())
print(tmp_state_dict["shared.0.weight"])

from tests.test_grpo import ActorCritic
import gymnasium as gym

env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = ActorCritic(state_dim, action_dim).to(device)

model.actor[0].load_state_dict({
    "weight": tmp_state_dict["shared.0.weight"],
    "bias": tmp_state_dict["shared.0.bias"]
})
model.actor[2].load_state_dict({
    "weight": tmp_state_dict["actor.0.weight"],
    "bias": tmp_state_dict["actor.0.bias"]
})
model.actor[4].load_state_dict({
    "weight": tmp_state_dict["actor.2.weight"],
    "bias": tmp_state_dict["actor.2.bias"]
})

odict_keys(['shared.0.weight', 'shared.0.bias', 'actor.0.weight', 'actor.0.bias', 'actor.2.weight', 'actor.2.bias', 'critic.0.weight', 'critic.0.bias', 'critic.2.weight', 'critic.2.bias'])
tensor([[-3.3123e-01, -2.5398e-01, -2.7501e-01,  3.7627e-01],
        [ 3.9722e-01,  1.0810e-01, -1.7589e-01, -3.2014e-01],
        [ 4.2063e-01, -2.2229e-01,  3.0371e-02,  3.1340e-01],
        [-1.6952e-01,  1.8697e-01, -3.1366e-01,  3.0789e-01],
        [-2.5695e-01,  1.6764e-01,  1.1338e-01,  6.5048e-02],
        [-1.1343e-01,  4.3815e-01, -3.8589e-01,  8.0395e-02],
        [ 3.0656e-02,  1.5751e-01,  4.4099e-01,  3.0622e-01],
        [-1.5366e-01, -1.5103e-01, -9.4528e-02, -3.7215e-02],
        [-2.1897e-01,  3.0419e-01,  4.6593e-01, -6.6638e-02],
        [-1.2995e-01, -2.9980e-01, -5.7404e-02,  6.1713e-02],
        [-2.8944e-01, -3.8228e-01,  3.2775e-01,  2.8904e-02],
        [ 2.1914e-01,  2.2931e-01,  3.8903e-01,  4.0974e-01],
        [ 3.0523e-01,  1.3249e-01,  1.8144e-01, -3.6944e-01],
     

<All keys matched successfully>

In [13]:
import gymnasium as gym

env = gym.make_vec("CartPole-v1", num_envs=32)
print(env.observation_space.shape[1])
print(env.action_space[0].n)


4
2
